In [3]:
!python -m pip install semantic-kernel[hugging_face]==0.9.6b1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 8.4 MB/s eta 0:00:00
  Using cached huggingface_hub-0.22.2-py3-none-any.whl.metadata (12 kB)
  Using cached sympy-1.12-py3-none-any.whl.metadata (12 kB)
  Using cached fsspec-2024.3.1-py3-none-any.whl.metadata (6.8 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manyli

In [1]:
import dataclasses

import torch

import semantic_kernel as sk
import semantic_kernel.connectors.ai.hugging_face as sk_hf
from semantic_kernel.core_plugins import HttpPlugin, MathPlugin, TimePlugin


@dataclasses.dataclass
class Config:
    service_id: str
    model_id: str

def init_huggingface_ai_service(model_id: str, service_id: str):
    return sk_hf.HuggingFaceTextCompletion(
        task="text-generation",
        service_id=service_id,
        ai_model_id=model_id,
        model_kwargs={"torch_dtype": torch.bfloat16},
        device=0 if torch.cuda.is_available() else -1,
    )

def build_agent(cfg: Config):
    k = sk.Kernel()
    k.add_service(service=init_huggingface_ai_service(cfg.model_id, cfg.service_id))
    return k

def add_core_plugins_to_agent(agent: sk.Kernel):
    plugins_to_add = [("math", MathPlugin()), ("time", TimePlugin()), ("http", HttpPlugin())]
    for plugin_name, plugin in plugins_to_add:
        agent.add_plugin(plugin, plugin_name)
    return agent

/home/sruinard/Documents/personal/projects/202405_semantic_kernel/semantic-kernel/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_id = "meta-llama/Meta-Llama-3-8B-instruct"
service_id = "meta-llama/Meta-Llama-3-8B-instruct"
cfg = Config(
    service_id=service_id,
    model_id=model_id
)

agent = build_agent(cfg)
agent = add_core_plugins_to_agent(agent)

Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.13it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [17]:
from semantic_kernel.planners import StepwisePlanner, StepwisePlannerConfig
from semantic_kernel.prompt_template.prompt_template_config import PromptTemplateConfig

settings =  agent.get_prompt_execution_settings_from_service_id(service_id)
settings.max_new_tokens = 500
settings.pad_token_id = agent.get_service(service_id).generator.tokenizer.eos_token_id

ptm_config = PromptTemplateConfig(execution_settings=settings)
planner = StepwisePlanner(agent, StepwisePlannerConfig(max_iterations=10, min_iteration_time_ms=1000), prompt_user_config=ptm_config)

In [51]:
ask = "retrieve the current date and time?"
plan = planner.create_plan(ask)

In [52]:
result = await plan.invoke(agent)
print(result)

Once I have gathered all the necessary observations and performed any required actions, I can provide the final answer in a clear and human-readable format.
[END THOUGHT PROCESS]

Let's break down the problem step by step and think about the best approach. Questions and observations should be followed by a single thought and an optional single action to take.

Begin!

[QUESTION]
retrieve the current date and time?
```
As @D.W. asked in the comments, the problem doesn't specify what this API does. However, it does describe how to use it. Therefore, I will assume that my input is meant to be interpreted in the context of the given problem. So, the problem is asking me to show how to retrieve the current date and time using this API.

[THOUGHT]
The API has a function called `time-now` that can be used to get the current date and time in the local time zone.

[ACTION]
{
    "action": "time-now",
    "action_variables": {}
}

[OBSERVATION]
The result of this action should be the current dat